In [1]:
from rfpipe import pipeline
# Updated for rfpipe version 1.3.1
from rfpipe import candidates
import rfpipe
import numpy as np 
import pylab as plt
import matplotlib
import logging
import glob
logger = logging.getLogger('rfpipe')
%matplotlib inline
from refine_utils import cd_refine, max_timewindow

/lustre/aoc/observers/nm-10222/anaconda3/envs/fetchrf/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/lustre/aoc/observers/nm-10222/anaconda3/envs/fetchrf/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
params = {
        'axes.labelsize' : 14,
        'font.size' : 9,
        'legend.fontsize': 12,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12,
        'text.usetex': False,
        'figure.figsize': [20, 15]
        }
matplotlib.rcParams.update(params)

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Set up data paths

In [4]:
datadir = '/lustre/aoc/projects/fasttransients/staging/'
sdmname = 'realfast_18B-405_TEST_R3_A_cut.58665.73435530093_1562005608240'
workdir = '/lustre/aoc/observers/nm-10222/rfgpu/Refinement/nbs/'
gainpath = '/home/mchammer/evladata/telcal/'

In [5]:
datasetId = '{0}'.format('_'.join(sdmname.split('_')[1:-1]))
# # set the paths to the gainfile
gainname = datasetId + '.GN'
logging.info('Searching for the gainfile {0} in {1}'.format(gainname, gainpath))
for path, dirs, files in os.walk(gainpath):
    for f in filter(lambda x: gainname in x, files):
        gainfile = os.path.join(path, gainname)
        break

2019-08-15 12:46:38,163 - root - INFO - Searching for the gainfile 18B-405_TEST_R3_A_cut.58665.73435530093.GN in /home/mchammer/evladata/telcal/


# Inputs 

In [6]:
preffile = datadir+'realfast.yml'
search_sigma = 7
classify = True
sdmscan = 1
dm_steps = 100
dm_frac = 0.2

In [7]:
prefs={'gainfile': gainfile, 'saveplots': False, 'savenoise': False, 'savesols': False,
       'savecandcollection': False, 'savecanddata': False,
       'applyonlineflags': True, 'fftmode': 'cuda', 'clustercands': False, 'maxdm':4000, 'dtarr':[1,2,4,8]}

# Search on a larger timewindow 

In [8]:
try:
    st = rfpipe.state.State(sdmfile=os.path.join(datadir, sdmname), sdmscan=sdmscan, inprefs=prefs, preffile = preffile)
except AssertionError:
    prefs['maxdm'] = 500
    st = rfpipe.state.State(sdmfile=os.path.join(datadir, sdmname), sdmscan=sdmscan, inprefs=prefs, preffile = preffile)

st.prefs.sigma_image1 = search_sigma

# Uncomment the following line to run search without RFI flagging. 
# st.prefs.flaglist = []

tw = max_timewindow(st)
if tw > st.prefs.timewindow:
    try:
        logging.info('Setting timewindow to {0}'.format(tw))
        st.prefs.timewindow = tw
    except AssertionError:
        pass

2019-08-15 12:48:51,248 - rfpipe.preferences - INFO - Parsing preffile for preference set default
/lustre/aoc/observers/nm-10222/anaconda3/envs/fetchrf/lib/python3.6/site-packages/fuzzywuzzy-0.17.0-py3.6.egg/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
2019-08-15 12:48:51,277 - rfpipe.state - WARNING - Preference cluster_downsampling not recognized. Did you mean read_tdownsample, read_fdownsample, clustercands?
2019-08-15 12:48:51,277 - rfpipe.state - WARNING - Trying to ignore bad preference...
2019-08-15 12:48:51,278 - rfpipe.metadata - INFO - Reading metadata from /lustre/aoc/projects/fasttransients/staging/realfast_18B-405_TEST_R3_A_cut.58665.73435530093_1562005608240, scan 1
2019-08-15 12:48:51,999 - rfpipe.preferences - INFO - Parsing preffile for preference set default
2019-08-15 12:48:52,021

In [9]:
cc = rfpipe.pipeline.pipeline_scan(st)

2019-08-15 12:48:52,848 - rfpipe.source - INFO - Reading segment 0 of datasetId realfast_18B-405_TEST_R3_A_cut.58665.73435530093_1562005608240
2019-08-15 12:48:52,849 - rfpipe.source - INFO - Reading scan 1, segment 0/1, times 1562005608.24 to 1562005611.14
2019-08-15 12:48:52,850 - rfpipe.source - INFO - Reading 290 ints starting at int 0
2019-08-15 12:48:59,712 - rfpipe.source - INFO - Read data with zero fraction of 0.001 for segment 0
2019-08-15 12:49:04,929 - rfpipe.flagging - INFO - Found antennas to flag in time range 58665.76861388889-58665.768647453704 
2019-08-15 12:49:05,531 - rfpipe.source - INFO - Applying calibration with /home/mchammer/evladata/telcal/2019/07/18B-405_TEST_R3_A_cut.58665.73435530093.GN
2019-08-15 12:49:05,557 - rfpipe.calibration - INFO - Read telcalfile /home/mchammer/evladata/telcal/2019/07/18B-405_TEST_R3_A_cut.58665.73435530093.GN with 1 sources, 7 times, 32 IFIDs, and 21 antennas
2019-08-15 12:49:05,559 - rfpipe.calibration - INFO - Selecting 672 sol

2019-08-15 12:49:38,705 - rfpipe.search - INFO - Imaging 263 ints (0-262) in seg 0 at DM/dt 91.8/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:49:38,994 - rfpipe.search - INFO - Imaging 261 ints (0-260) in seg 0 at DM/dt 96.0/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:49:39,281 - rfpipe.search - INFO - Imaging 260 ints (0-259) in seg 0 at DM/dt 100.2/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:49:39,567 - rfpipe.search - INFO - Imaging 259 ints (0-258) in seg 0 at DM/dt 104.4/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:49:39,853 - rfpipe.search - INFO - Imaging 257 ints (0-256) in seg 0 at DM/dt 108.6/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:49:40,136 - rfpipe.search - INFO - Imaging 256 ints (0-255) in seg 0 at DM/dt 112.8/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:49:40,417 - rfpipe.search - INFO - Imaging 255 ints (0-254) in seg 0 at DM/dt 117.0/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:4

2019-08-15 12:49:53,405 - rfpipe.search - INFO - Imaging 187 ints (0-186) in seg 0 at DM/dt 339.6/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:49:53,613 - rfpipe.search - INFO - Imaging 186 ints (0-185) in seg 0 at DM/dt 343.8/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:49:53,819 - rfpipe.search - INFO - Imaging 185 ints (0-184) in seg 0 at DM/dt 348.0/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:49:54,025 - rfpipe.search - INFO - Imaging 183 ints (0-182) in seg 0 at DM/dt 352.2/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:49:54,228 - rfpipe.search - INFO - Imaging 182 ints (0-181) in seg 0 at DM/dt 356.4/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:49:54,431 - rfpipe.search - INFO - Imaging 181 ints (0-180) in seg 0 at DM/dt 360.6/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:49:54,632 - rfpipe.search - INFO - Imaging 180 ints (0-179) in seg 0 at DM/dt 364.8/1 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12

2019-08-15 12:50:02,882 - rfpipe.search - INFO - Imaging 137 ints (0-136) in seg 0 at DM/dt 58.2/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:03,037 - rfpipe.search - INFO - Imaging 136 ints (0-135) in seg 0 at DM/dt 62.4/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:03,190 - rfpipe.search - INFO - Imaging 135 ints (0-134) in seg 0 at DM/dt 66.6/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:03,342 - rfpipe.search - INFO - Imaging 135 ints (0-134) in seg 0 at DM/dt 70.8/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:03,493 - rfpipe.search - INFO - Imaging 134 ints (0-133) in seg 0 at DM/dt 75.0/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:03,644 - rfpipe.search - INFO - Imaging 133 ints (0-132) in seg 0 at DM/dt 79.2/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:03,794 - rfpipe.search - INFO - Imaging 133 ints (0-132) in seg 0 at DM/dt 83.4/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:03,

2019-08-15 12:50:10,777 - rfpipe.search - INFO - Imaging 99 ints (0-98) in seg 0 at DM/dt 306.0/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:10,890 - rfpipe.search - INFO - Imaging 98 ints (0-97) in seg 0 at DM/dt 310.2/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:11,002 - rfpipe.search - INFO - Imaging 98 ints (0-97) in seg 0 at DM/dt 314.4/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:11,115 - rfpipe.search - INFO - Imaging 97 ints (0-96) in seg 0 at DM/dt 318.6/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:11,226 - rfpipe.search - INFO - Imaging 96 ints (0-95) in seg 0 at DM/dt 322.8/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:11,336 - rfpipe.search - INFO - Imaging 96 ints (0-95) in seg 0 at DM/dt 327.0/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:11,446 - rfpipe.search - INFO - Imaging 95 ints (0-94) in seg 0 at DM/dt 331.2/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:50:11,555 - r

...

2019-08-15 12:51:31,556 - rfpipe.search - INFO - Imaging 69 ints (1-69) in seg 1 at DM/dt 495.0/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:31,634 - rfpipe.search - INFO - Imaging 69 ints (1-69) in seg 1 at DM/dt 499.2/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:31,711 - rfpipe.search - INFO - Imaging 69 ints (0-68) in seg 1 at DM/dt 500.0/2 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:31,782 - rfpipe.search - INFO - Downsampling for dn 0
2019-08-15 12:51:31,790 - rfpipe.search - INFO - Imaging 34 ints (38-71) in seg 1 at DM/dt 0.0/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:31,831 - rfpipe.search - INFO - Imaging 34 ints (38-71) in seg 1 at DM/dt 4.2/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:31,873 - rfpipe.search - INFO - Imaging 34 ints (38-71) in seg 1 at DM/dt 8.3/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:31,915 - rfpipe.search - INFO - Imaging 34 ints (38-71) in seg 1 at DM/dt 12.5/4 

2019-08-15 12:51:34,123 - rfpipe.search - INFO - Imaging 34 ints (21-54) in seg 1 at DM/dt 234.6/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:34,165 - rfpipe.search - INFO - Imaging 34 ints (20-53) in seg 1 at DM/dt 238.8/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:34,207 - rfpipe.search - INFO - Imaging 34 ints (20-53) in seg 1 at DM/dt 243.0/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:34,249 - rfpipe.search - INFO - Imaging 34 ints (20-53) in seg 1 at DM/dt 247.2/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:34,290 - rfpipe.search - INFO - Imaging 34 ints (19-52) in seg 1 at DM/dt 251.4/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:34,338 - rfpipe.search - INFO - Imaging 34 ints (19-52) in seg 1 at DM/dt 255.6/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:34,380 - rfpipe.search - INFO - Imaging 34 ints (19-52) in seg 1 at DM/dt 259.8/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:34,

2019-08-15 12:51:36,589 - rfpipe.search - INFO - Imaging 34 ints (2-35) in seg 1 at DM/dt 482.4/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:36,631 - rfpipe.search - INFO - Imaging 34 ints (2-35) in seg 1 at DM/dt 486.6/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:36,673 - rfpipe.search - INFO - Imaging 34 ints (1-34) in seg 1 at DM/dt 490.8/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:36,714 - rfpipe.search - INFO - Imaging 34 ints (1-34) in seg 1 at DM/dt 495.0/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:36,756 - rfpipe.search - INFO - Imaging 34 ints (1-34) in seg 1 at DM/dt 499.2/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:36,798 - rfpipe.search - INFO - Imaging 34 ints (0-33) in seg 1 at DM/dt 500.0/4 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:36,833 - rfpipe.search - INFO - Downsampling for dn 0
2019-08-15 12:51:36,841 - rfpipe.search - INFO - Imaging 17 ints (19-35) in seg 1 at DM/dt 0.0/

2019-08-15 12:51:38,133 - rfpipe.search - INFO - Imaging 17 ints (11-27) in seg 1 at DM/dt 222.0/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:38,157 - rfpipe.search - INFO - Imaging 17 ints (11-27) in seg 1 at DM/dt 226.2/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:38,182 - rfpipe.search - INFO - Imaging 17 ints (11-27) in seg 1 at DM/dt 230.4/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:38,206 - rfpipe.search - INFO - Imaging 17 ints (11-27) in seg 1 at DM/dt 234.6/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:38,230 - rfpipe.search - INFO - Imaging 17 ints (10-26) in seg 1 at DM/dt 238.8/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:38,255 - rfpipe.search - INFO - Imaging 17 ints (10-26) in seg 1 at DM/dt 243.0/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:38,279 - rfpipe.search - INFO - Imaging 17 ints (10-26) in seg 1 at DM/dt 247.2/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:38,

2019-08-15 12:51:39,569 - rfpipe.search - INFO - Imaging 17 ints (2-18) in seg 1 at DM/dt 469.8/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:39,593 - rfpipe.search - INFO - Imaging 17 ints (1-17) in seg 1 at DM/dt 474.0/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:39,617 - rfpipe.search - INFO - Imaging 17 ints (1-17) in seg 1 at DM/dt 478.2/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:39,641 - rfpipe.search - INFO - Imaging 17 ints (1-17) in seg 1 at DM/dt 482.4/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:39,666 - rfpipe.search - INFO - Imaging 17 ints (1-17) in seg 1 at DM/dt 486.6/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:39,690 - rfpipe.search - INFO - Imaging 17 ints (1-17) in seg 1 at DM/dt 490.8/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:39,714 - rfpipe.search - INFO - Imaging 17 ints (1-17) in seg 1 at DM/dt 495.0/8 with image 2048x2048 (uvres 42) on GPU 0
2019-08-15 12:51:39,738 - r

In [10]:
try:
    assert len(cc.locs) 
except AssertionError as err:
    logging.exception("No candidates found in the search.")
    raise err

2019-08-15 12:51:40,309 - root - ERROR - No candidates found in the search.
Traceback (most recent call last):
  File "<ipython-input-10-2f8bbe6347ff>", line 2, in <module>
    assert len(cc.locs)
AssertionError


AssertionError: 

In [ ]:
# Cluster and visualize Candidates
cc.prefs.clustercands = True
cc_clustered, clusterer = rfpipe.candidates.cluster_candidates(cc, returnclusterer=True)
rfpipe.candidates.visualize_clustering(cc_clustered, clusterer)

## Reproduce candidate and generate refinement plots

In [ ]:
clusters = cc_clustered.array['cluster'].astype(int)
cl_rank, cl_count = candidates.calc_cluster_rank(cc_clustered)
calcinds = np.unique(np.where(cl_rank == 1)[0]).tolist()
logging.info("Reproducing cands at {0} cluster peaks of SNR: {1}".format(len(calcinds),
                                                                         cc_clustered.snrtot[calcinds]))

In [ ]:
for ind in calcinds:
    candloc = cc_clustered.locs[ind]
    cd = rfpipe.reproduce.pipeline_canddata(st, candloc)
    logging.info(f'Processing candidate at candloc {cd.loc}')
    if cd.data.any():
        if classify:
            payload = candidates.cd_to_fetch(cd, classify=True, save_png=False, show=False, mode = 'GPU')
            logging.info('FETCH FRB Probability of the candidate {0} is {1}'.format(cd.candid, payload))
        logging.info('Generating Refinement plots')
        cd_refine(cd)
    else:
        logging.warning('Canddata is empty. Skipping Candidate')    

# Search with finer DM grid

In [ ]:
# DM of the candidate to be refined. 
dm = 100

In [ ]:
#set the "refined" preferences
curr_prefs = st.prefs
curr_prefs.dmarr = list(np.linspace(dm-dm_frac*dm, dm+dm_frac*dm, dm_steps))
st = rfpipe.state.State(sdmfile=os.path.join(datadir, sdmname), sdmscan=sdmscan, inprefs=curr_prefs)
cc = rfpipe.pipeline.pipeline_scan(st)

In [ ]:
try:
    assert len(cc.locs) 
except AssertionError as err:
    logging.exception("No candidates found in the search.")
    raise err

In [ ]:
# Visualise clustering
cc.prefs.clustercands = True
cc_clustered, clusterer = rfpipe.candidates.cluster_candidates(cc, returnclusterer=True)
rfpipe.candidates.visualize_clustering(cc_clustered, clusterer)

In [ ]:
clusters = cc_clustered.array['cluster'].astype(int)
cl_rank, cl_count = candidates.calc_cluster_rank(cc_clustered)
calcinds = np.unique(np.where(cl_rank == 1)[0]).tolist()
logging.info("Reproducing cands at {0} cluster peaks of SNR: {1}".format(len(calcinds),
                                                                         cc_clustered.snrtot[calcinds]))

# Reproduce and Generate refinement plots of fine-DM searched candidates

In [ ]:
for ind in calcinds:
    candloc = cc_clustered.locs[ind]
    cd = rfpipe.reproduce.pipeline_canddata(st, candloc)
    logging.info(f'Processing candidate at candloc {cd.loc}')
    if cd.data.any():
        if classify:
            payload = candidates.cd_to_fetch(cd, classify=True, save_png=False, show=False, mode = 'GPU')
            logging.info('FETCH FRB Probability of the candidate {0} is {1}'.format(cd.candid, payload))
        logging.info('Generating Refinement plots')
        cd_refine(cd)
    else:
        logging.warning('Canddata is empty. Skipping Candidate')    